<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

In [2]:
import folium
import wget
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Mission Outcome,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),Success,No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']] #Take Important Parameters

spacex_df_class = spacex_df[['Launch Site','class']] #To find class average (Successful Rate)

classavg = spacex_df_class.groupby('Launch Site',as_index=False).mean()

classavg.rename({'class': 'Success Rate'}, axis=1, inplace=True)

print(classavg)

Launch_Site = spacex_df[['Launch Site', 'Lat', 'Long']].groupby('Launch Site', as_index=False).first()

print('')

print("Merging Tables")

print('')


print(Launch_Site)

del classavg['Launch Site']

Launch_Site = Launch_Site.join(classavg)

print('')

print("Merging Completed")


Launch_Site

    Launch Site  Success Rate
0   CCAFS LC-40      0.269231
1  CCAFS SLC-40      0.428571
2    KSC LC-39A      0.769231
3   VAFB SLC-4E      0.400000

Merging Tables

    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610746

Merging Completed


,Launch Site,Lat,Long,Success Rate
0,CCAFS LC-40,28.562302,-80.577356,0.269231
1,CCAFS SLC-40,28.563197,-80.576820,0.428571
2,KSC LC-39A,28.573255,-80.646895,0.769231
3,VAFB SLC-4E,34.632834,-120.610746,0.400000


In [6]:
SiteOne = Launch_Site.iloc[0,1:3]
SiteTwo = Launch_Site.iloc[1,1:3]
SiteThree = Launch_Site.iloc[2,1:3]
SiteFour = Launch_Site.iloc[3,1:3]
## Make them lists 
SiteOne = SiteOne.tolist()
SiteTwo = SiteTwo.tolist()
SiteThree = SiteThree.tolist()
SiteFour = SiteFour.tolist()
print(SiteOne)
print(SiteTwo)
print(SiteThree)
print(SiteFour)

[28.56230196799018, -80.57735647504778]
[28.563197177407144, -80.57682003124195]
[28.57325457037815, -80.64689528960382]
[34.632834161782775, -120.61074553068327]


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
#Site One
circle1 = folium.Circle(SiteOne, radius=50, color='#d35400', fill=True).add_child(folium.Popup('CCAFS LC-40'))
# Create a red circle 
marker1 = folium.map.Marker(
    SiteOne,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS LC-40',
        )
    )
site_map.add_child(circle1)
site_map.add_child(marker1)
#Site Two
circle2 = folium.Circle(SiteTwo, radius=50, color='#d35400', fill=True).add_child(folium.Popup('CCAFS SLC-40'))
# Create a red circle 
marker2 = folium.map.Marker(
    SiteTwo,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'CCAFS SLC-40',
        )
    )
site_map.add_child(circle2)
site_map.add_child(marker2)
#Site Three
circle3 = folium.Circle(SiteThree, radius=50, color='#d35400', fill=True).add_child(folium.Popup('KSC LC-39A'))
# Create a red circle 
marker3 = folium.map.Marker(
    SiteThree,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'KSC LC-39A',
        )
    )
site_map.add_child(circle3)
site_map.add_child(marker3)
#Site Four 
circle4 = folium.Circle(SiteFour, radius=50, color='#d35400', fill=True).add_child(folium.Popup('VAFB SLC-4E'))
# Create a red circle 
marker4 = folium.map.Marker(
    SiteFour,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'VAFB SLC-4E',
        )
    )
site_map.add_child(circle4)
site_map.add_child(marker4)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


**In short, all site locations are near the coast and the Equator line, Space X focuses on locations close to water and 0 latitude.**

# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(8)

,Launch Site,Lat,Long,class
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [11]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [12]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
marker_color = []
for x in spacex_df['class']:
    if x == 1:
        marker_color.append('green')
    else:
        marker_color.append('red')
        
spacex_df['marker_color'] = marker_color
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [13]:
# Function to assign color to launch outcome #Same As above (Covered!)
# def assign_marker_color(launch_outcome):
#     if launch_outcome == 1:
#         return 'green'
#     else:
#         return 'red'
    
# spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
# spacex_df.tail(10)

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add the Marker cluster to the site map
site_map.add_child(marker_cluster)
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for i in range(spacex_df.shape[0]):

    marker = folium.Marker(location = (spacex_df.iloc[i]['Lat'],spacex_df.iloc[i]['Long']), icon = folium.Icon(color='white', icon_color =spacex_df.iloc[i]['marker_color'])).add_to(marker_cluster)

site_map.add_child(marker_cluster)


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [16]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest railway using MousePosition and calculate the distance between the railway point to the launch site.


In [17]:
###CCAFS LC-40 is SiteOne
lat1 = SiteOne[0]
lon1 = SiteOne[1]
###Nearet railway coordinates are lat = [28.57197] long = [-80.58524]
lat2 = 28.57197
lon2 = -80.58524
###Calculating the distance
distance_railway = calculate_distance(lat1, lon1, lat2, lon2)
distance_railway

1.3226969527963404

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [18]:
# create and add a folium.Marker on your selected closest raiwaly point on the map
# show the distance to the launch site using the icon property 

rail_circle = folium.Circle([lat2,lon2], radius=200, color='#d35400', fill=True).add_child(folium.Popup('Distance ≈ 1.32'))
# Create a red circle 
rail_marker = folium.map.Marker(
    [lat2,lon2],
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Closest Rail way',
        )
    )
site_map.add_child(rail_circle)
site_map.add_child(rail_marker)



*TODO:* Draw a `PolyLine` between a launch site to the selected


In [19]:
# Create a `folium.PolyLine` object using the railway point coordinate and launch site coordinate
locations=[[lat1,lon1],[lat2,lon2]]
PolyLine =folium.PolyLine(locations,weight=5)
site_map.add_child(PolyLine)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, coastline, highway, etc.


In [20]:
# Create a marker with distance to a closest city, coastline, highway, etc.

#Closest City is Orlando 

Orlando_Location = [28.523,-81.3826]

#Closest coastline is on the east side

Coastline_Location = [28.56146,-80.56746]

#Closest highway is on the west side

Highway_Location = [28.5627,-80.58703]

##Calculating Distances

lat3 = Orlando_Location[0]
lon3 = Orlando_Location[1]
lat4 = Coastline_Location[0]
lon4 = Coastline_Location[1]
lat5 = Highway_Location[0]
lon5 = Highway_Location[1]

distance_city = calculate_distance(lat1, lon1, lat3, lon3)
distance_coastline = calculate_distance(lat1, lon1, lat4, lon4)
distance_highway = calculate_distance(lat1, lon1, lat5, lon5)

print('City Distance ≈', distance_city)
print('Coast Line Distance ≈', distance_coastline)
print('Highway Distance ≈', distance_highway)

## Marking the highlighted locations with circles 

##Closest City Mapping
city_circle = folium.Circle(Orlando_Location, radius=8000, color='#d35400', fill=True).add_child(folium.Popup('Orlando'))
# Create a red circle 
city_marker = folium.map.Marker(
    Orlando_Location,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Orlando',
        )
    )
site_map.add_child(city_circle)
site_map.add_child(city_marker)

#Closest Coastline Mapping

coast_circle = folium.Circle(Coastline_Location, radius=200, color='#d35400', fill=True).add_child(folium.Popup('East Coast'))
# Create a red circle 
coast_marker = folium.map.Marker(
    Coastline_Location,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Coast Line',
        )
    )
site_map.add_child(coast_circle)
site_map.add_child(coast_marker)

#Cloasest Highway Mapping

highway_circle = folium.Circle(Highway_Location, radius=200, color='#d35400', fill=True).add_child(folium.Popup('Highway'))
# Create a red circle 
highway_marker = folium.map.Marker(
    Highway_Location,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Highway',
        )
    )
site_map.add_child(highway_circle)
site_map.add_child(highway_marker)

# Draw a line between the marker to the launch site

Location_City=[[lat1,lon1],[lat3,lon3]]
PolyLine =folium.PolyLine(Location_City,weight=5)
site_map.add_child(PolyLine)
Location_Coast=[[lat1,lon1],[lat4,lon4]]
PolyLine =folium.PolyLine(Location_Coast,weight=5)
site_map.add_child(PolyLine)
Location_Highway=[[lat1,lon1],[lat5,lon5]]
PolyLine =folium.PolyLine(Location_Highway,weight=5)
site_map.add_child(PolyLine)


City Distance ≈ 78.80246522405488
Coast Line Distance ≈ 0.9713445842775478
Highway Distance ≈ 0.9460695157584161


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


**1 - Yes**

**2 - Yes**

**3 - Yes**

**4 - Yes**


**In short, being close to railways and highways would help a lot in transportation cost-reduction.
Being close to coastline and away from cities adds great safety value to the rockets experiments.**

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
